In [2]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
from pyod.models.iforest import IForest  # Modelo Isolation Forest para detección de anomalías

# Configuración de conexión a la base de datos
DB_CONFIG = {
    "user": "root",
    "password": "1234",
    "host": "127.0.0.1",
    "port": "3306",
    "database": "gp-mlp-telemtry"
}

# Crear la cadena de conexión
DATABASE_URL = f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(DATABASE_URL)

# Función para ejecutar el procedimiento almacenado y obtener datos
def obtener_datos(estacion_id, sensor_id, fecha):
    fecha_consulta = pd.to_datetime(fecha).strftime("%Y-%m-%d")
    
    with engine.connect() as conn:
        result = conn.execute(text("CALL `gp-mlp-telemtry`.GetSensorData(:estacion, :sensor, :periodo)"), {
            "estacion": estacion_id,
            "sensor": sensor_id,
            "periodo": 2  # Ajustado a un valor válido
        })
        datos = result.fetchall()
        columnas = result.keys()
    
    # Convertir a DataFrame
    df = pd.DataFrame(datos, columns=columnas)
    
    # Filtrar solo los datos hasta la fecha ingresada
    df = df[df["fecha_hora"] <= fecha_consulta]
    
    return df

# Función para detectar anomalías con PyOD
def detectar_anomalias(datos):
    if len(datos) < 10:
        print("No hay suficientes datos para detección de anomalías.")
        return pd.DataFrame()

    # Convertir datos a matriz NumPy para el modelo
    X = datos["valor"].values.reshape(-1, 1)

    # Modelo de PyOD: Isolation Forest
    modelo = IForest(contamination=0.1)  # Ajustar 'contamination' según necesidad
    modelo.fit(X)

    # Obtener predicciones
    datos["anomaly"] = modelo.predict(X)

    # Filtrar anomalías
    anomalías = datos[datos["anomaly"] == 1]  # 1 indica anomalía en PyOD
    
    return anomalías

if __name__ == "__main__":
    fecha_revision = input("Ingrese la fecha de revisión (YYYY-MM-DD): ")
    estacion_id = int(input("Ingrese el ID de la estación: "))
    sensor_id = int(input("Ingrese el ID del sensor: "))

    # Obtener datos llamando al procedimiento almacenado
    datos = obtener_datos(estacion_id, sensor_id, fecha_revision)

    if datos.empty:
        print("No se encontraron datos en la base de datos.")
    else:
        # Detectar anomalías
        alertas = detectar_anomalias(datos)

        if not alertas.empty:
            print("\n🔴 ALERTA: Se detectaron anomalías en los últimos 7 días.")
            print(alertas[["fecha_hora", "valor"]])
        else:
            print("\n✅ No se detectaron anomalías en los últimos 7 días.")



Ingrese la fecha de revisión (YYYY-MM-DD):  2025-03-16
Ingrese el ID de la estación:  114
Ingrese el ID del sensor:  110



🔴 ALERTA: Se detectaron anomalías en los últimos 7 días.
            fecha_hora  valor
5  2025-03-13 05:00:00   7.15
7  2025-03-13 07:00:00   7.10
28 2025-03-14 04:00:00   7.18
30 2025-03-14 06:00:00   7.14
31 2025-03-14 07:00:00   7.11
32 2025-03-14 08:00:00   7.11
54 2025-03-15 06:00:00   7.13
